In [1]:
import math
from itertools import combinations, permutations

import pandas as pd

from lol_fandom import get_tournaments
from lol_fandom import get_scoreboard_games


pd.set_option('display.max_columns', None)

In [2]:
class Team:
    """Team information"""
    q = math.log(10) / 400

    def __init__(self, name, league):
        self.name = name
        self.league = league
        self.win = 0
        self.loss = 0
        self.streak = 0
        self.r = 1000
        self.RD = 350
        self.last_game_date = None

    def update_team_name(self, name):
        self.name = name

    def update_league(self, league):
        self.league = league

    def update_last_game_date(self, game_date):
        self.last_game_date = game_date

    def update_streak(self, result):
        result = 1 if result == 1 else -1
        if (self.streak > 0) == (result > 0):
            self.streak += result
        else:
            self.streak = result

    @classmethod
    def get_g(cls, RDi):
        """Compute g(RDi)

        Args:
            RDi (float): Ratings Deviation (RD)

        Returns:
            float: g(RDi)
        """
        return 1 / math.sqrt(1 + (3 * cls.q ** 2 * RDi ** 2) / math.pi ** 2)

    @classmethod
    def get_e(cls, r0, ri, g):
        """Compute E(s|r0, ri, RDi)

        Args:
            r0 (float): previous rating
            ri (float): rating of opponent
            g (float): g(RDi)

        Returns:
            float: E(s | r0, ri, RDi)
        """
        return 1 / (1 + 10 ** ((g * (r0 - ri)) / -400))

    @classmethod
    def get_d(cls, g, e):
        """Compute d^2

        Args:
            g (float): g(RDi)
            e (float): E(s | r0, ri, RDi)

        Returns:
            float: d^2
        """
        return 1 / (cls.q ** 2 * g ** 2 * e * (1 - e))

    def init_rd(self):
        """Initialize RD"""
        self.RD = 350

    @classmethod
    def update_point(cls, team1, team2, result):
        """Update ratings of team1 and team2

        Args:
            team1 (Team): Team1
            team2 (Team): Team2
            result (int): 1 is Team1 win, 0 is Team2 win
        """
        assert isinstance(team1, Team)
        assert isinstance(team2, Team)

        team1_r = team1.r
        team2_r = team2.r
        team1_RD = team1.RD
        team2_RD = team2.RD

        team1._update_point(team2_r, team2_RD, result)
        team2._update_point(team1_r, team1_RD, 1 - result)

    def _update_point(self, ri, RDi, s):
        if s == 1:
            self.win += 1
        else:
            self.loss += 1

        g_RD = Team.get_g(RDi)
        e = Team.get_e(self.r, ri, g_RD)
        d_2 = Team.get_d(g_RD, e)
        self.r = self.r + Team.q / (1 / self.RD ** 2 + 1 / d_2) * g_RD * (s - e)

        self.RD = math.sqrt((1 / self.RD ** 2 + 1 / d_2) ** -1)

    def get_win_prob(self, opponent):
        """Get win probability

        Args:
            opponent (Team): Opponent team

        Returns:
            float: Win probability
        """
        return Team.get_e(self.r, opponent.r, Team.get_g(opponent.RD))

    def to_dict(self):
        """Make Team class to dictionary

        Returns:
            dict: Dictionary of Team instance
        """
        data = {
            'Team': self.name,
            'League': self.league,
            'Win': self.win,
            'Loss': self.loss,
            'WinRate': self.win / (self.win + self.loss) if self.win != 0 else 0,
            'Streak': self.streak,
            'r': self.r,
            'RD': self.RD,
            'last_game_date': self.last_game_date,
        }

        return data

def get_team_id(teams_id, name):
    return teams_id.loc[teams_id['team'] == name, 'team_id'].iloc[0]

def proceed_rating(teams_id, teams, games):
    """Proceed rating with teams and games

    Args:
        teams_id (DataFrame): ID of teams 
        teams (list): List of Team instances
        games (DataFrame): Scoreboard games
    """
    for row in games.itertuples():
        team1, team2 = row.Team1, row.Team2
        game_date = row._6
        result = 1 if row.WinTeam == team1 else 0
        id1, id2 = get_team_id(teams_id, team1), get_team_id(teams_id, team2)
        Team.update_point(teams[id1], teams[id2], result)
        teams[id1].update_last_game_date(game_date)
        teams[id2].update_last_game_date(game_date)
        teams[id1].update_streak(result)
        teams[id2].update_streak(1 - result)

def get_rating(teams):
    """Get rating of teams

    Args:
        teams (list): List of Team instances

    Returns:
        DataFrame: Rating of teams
    """
    ratings = pd.DataFrame(data=map(lambda x: x.to_dict(), teams.values()))
    ratings = ratings.sort_values(by='r', ascending=False).reset_index(drop=True)
    return ratings

def get_team_name(same_team_names, name):
    """Get latest name of the team

    Args:
        same_team_names (dict): Dictionary of names of same teams
        name (str): Name of the team

    Returns:
        str: Latest name of the team
    """
    while name in same_team_names:
        name = same_team_names[name]
    return name

def is_proper_league(league):
    if 'WCS' == league or 'MSI' == league:
        return False
    return True

In [3]:
teams_id = pd.read_csv('./csv/teams_id.csv')
teams_id

,team,team_id
0,Azubu Frost,1
1,CJ Entus Frost,1
2,MiG Frost,1
3,MKZ,2
4,Little Hippo,3
...,...,...
717,Six Karma,602
718,FUT Esports,603
719,FENNEL,604
720,HELL PIGS,605


In [4]:
teams = {}
for year in range(2011, 2024):
    tournaments = pd.read_csv(f'./csv/tournaments/{year}_tournaments.csv')
    scoreboard_games = pd.read_csv(f'./csv/scoreboard_games/{year}_scoreboard_games.csv')
    for page in tournaments['OverviewPage']:
        print(f'{page} rating ...')
        sg = scoreboard_games.loc[scoreboard_games['OverviewPage'] == page]
        print(f'\t{sg.shape[0]} matches')

        league = sg['League'].iloc[0]
        team_names = sg[['Team1', 'Team2']].unstack().unique()
        team_check = True
        for name in team_names:
            if name not in teams_id['team'].values:
                print(f'{name} not in teams')
                team_check = False
                break
            id = get_team_id(teams_id, name)
            if id not in teams:
                teams[id] = Team(name, league)
            else:
                teams[id].update_team_name(name)
                if is_proper_league(league):
                    teams[id].update_league(league)
        if not team_check:
            break
        
        for name in team_names:
            id = get_team_id(teams_id, name)
            teams[id].init_rd()

        proceed_rating(teams_id, teams, sg)

    if not team_check:
        break
if team_check:
    rating = get_rating(teams)

Season 1 World Championship rating ...
	28 matches
Champions/2012 Season/Spring rating ...
	44 matches
DreamHack Summer 2012 rating ...
	16 matches
Champions/2012 Season/Summer rating ...
	46 matches
Season 2/Regional Finals/Europe rating ...
	17 matches
Season 2/Regional Finals/Korea rating ...
	16 matches
Season 2 World Championship rating ...
	31 matches
Champions/2013 Season/Winter rating ...
	112 matches
NA LCS/Season 3/Spring Season rating ...
	112 matches
EU LCS/Season 3/Spring Season rating ...
	112 matches
LPL/2013 Season/Spring Season rating ...
	112 matches
Champions/2013 Season/Spring rating ...
	87 matches
EU LCS/Season 3/Spring Playoffs rating ...
	15 matches
NA LCS/Season 3/Spring Playoffs rating ...
	20 matches
EU LCS/Season 3/Summer Promotion rating ...
	28 matches
NA LCS/Season 3/Summer Promotion rating ...
	24 matches
NA LCS/Season 3/Summer Season rating ...
	113 matches
EU LCS/Season 3/Summer Season rating ...
	118 matches
LPL/2013 Season/Spring Playoffs rating ...


In [5]:
rating['last_game_date'] = pd.to_datetime(rating['last_game_date'])

In [6]:
rating

,Team,League,Win,Loss,WinRate,Streak,r,RD,last_game_date
0,Gen.G,LCK,566,386,0.594538,2,2174.436044,125.359783,2023-02-03 12:32:00
1,T1,LCK,857,439,0.661265,1,2104.146209,145.485465,2023-02-04 11:28:00
2,Dplus KIA,LCK,415,172,0.706985,1,2053.279859,139.358524,2023-02-05 10:45:00
3,Samsung White,LTC,94,42,0.691176,1,2009.034081,129.125220,2014-10-19 09:44:00
4,Liiv SANDBOX,LCK,197,213,0.480488,-2,1955.995677,107.867098,2023-02-03 12:32:00
...,...,...,...,...,...,...,...,...,...
601,Bencheados,CLS,7,31,0.184211,-6,325.064140,300.660297,2017-03-30 23:23:00
602,V3 Esports,LJL,117,145,0.446565,-6,321.256428,250.201348,2023-02-05 12:36:00
603,Flash Wolves,RR,403,292,0.579856,-78,287.526103,308.107825,2019-09-07 09:01:00
604,Gravitas,LCO,37,124,0.229814,-22,212.771879,170.229248,2022-08-09 09:16:00


In [7]:
rating.loc[(rating['League'] == 'LCK') & (rating['last_game_date'].dt.year == 2023)]

,Team,League,Win,Loss,WinRate,Streak,r,RD,last_game_date
0,Gen.G,LCK,566,386,0.594538,2,2174.436044,125.359783,2023-02-03 12:32:00
1,T1,LCK,857,439,0.661265,1,2104.146209,145.485465,2023-02-04 11:28:00
2,Dplus KIA,LCK,415,172,0.706985,1,2053.279859,139.358524,2023-02-05 10:45:00
4,Liiv SANDBOX,LCK,197,213,0.480488,-2,1955.995677,107.867098,2023-02-03 12:32:00
7,Hanwha Life Esports,LCK,402,410,0.495074,2,1886.433809,121.846096,2023-02-04 07:53:00
9,KT Rolster,LCK,590,410,0.590000,2,1801.426100,113.106740,2023-02-05 07:14:00
13,Nongshim RedForce,LCK,109,161,0.403704,-1,1722.893817,133.354334,2023-02-04 11:28:00
14,Kwangdong Freecs,LCK,413,457,0.474713,-1,1719.135733,142.812855,2023-02-05 10:45:00
16,BRION,LCK,170,288,0.371179,-2,1676.607067,110.831086,2023-02-05 07:14:00
17,DRX,LCK,564,505,0.527596,-2,1676.180503,108.654029,2023-02-04 07:53:00


In [8]:
rating.loc[(rating['League'] == 'LPL') & (rating['last_game_date'].dt.year == 2023)]

,Team,League,Win,Loss,WinRate,Streak,r,RD,last_game_date
5,JD Gaming,LPL,381,284,0.572932,3,1949.536544,246.393739,2023-02-02 09:58:00
6,LNG Esports,LPL,389,397,0.494911,6,1924.986802,183.182911,2023-02-05 10:42:00
8,EDward Gaming,LPL,707,439,0.616928,2,1823.820602,181.522192,2023-02-05 12:10:00
10,Invictus Gaming,LPL,641,482,0.570793,-1,1763.713398,170.616805,2023-02-04 10:59:00
12,Top Esports,LPL,392,299,0.567294,1,1727.496595,159.788818,2023-02-05 08:52:00
20,Bilibili Gaming,LPL,225,227,0.497788,1,1662.589835,158.143521,2023-02-04 14:19:00
24,Oh My God,LPL,445,530,0.456410,4,1626.231910,178.372987,2023-02-04 08:09:00
26,Weibo Gaming,LPL,280,241,0.537428,-1,1615.346603,172.819905,2023-02-04 14:19:00
35,ThunderTalk Gaming,LPL,110,200,0.354839,1,1517.729544,194.476199,2023-02-04 10:59:00
38,Rare Atom,LPL,293,392,0.427737,-4,1514.799982,149.474497,2023-02-05 10:42:00


In [9]:
rating.loc[(rating['League'] == 'LEC') & (rating['last_game_date'].dt.year == 2023)]

,Team,League,Win,Loss,WinRate,Streak,r,RD,last_game_date
27,Team Vitality,LEC,158,200,0.441341,1,1568.156794,173.103011,2023-02-05 18:06:00
30,G2 Esports,LEC,546,417,0.566978,2,1552.696280,168.076217,2023-02-05 21:23:00
32,MAD Lions,LEC,306,313,0.494346,2,1540.443835,171.210315,2023-02-05 19:24:00
48,SK Gaming,LEC,173,197,0.467568,1,1450.038273,189.827390,2023-02-05 17:06:00
71,Astralis,LEC,214,218,0.495370,3,1338.657559,181.808266,2023-02-05 20:29:00
80,Team BDS,LEC,11,33,0.250000,-2,1309.653657,159.166784,2023-02-05 18:06:00
97,Team Heretics,LEC,4,4,0.500000,-1,1266.110954,157.922502,2023-02-05 19:24:00
120,KOI (Spanish Team),LEC,3,5,0.375000,-1,1204.688639,173.004440,2023-02-05 21:23:00
165,Fnatic,LEC,752,475,0.612877,-3,1140.392370,152.990555,2023-02-05 20:29:00
265,Excel Esports,LEC,62,101,0.380368,-5,976.262296,158.666795,2023-02-05 17:06:00


In [10]:
rating.loc[(rating['League'] == 'LCS') & (rating['last_game_date'].dt.year == 2023)]

,Team,League,Win,Loss,WinRate,Streak,r,RD,last_game_date
18,FlyQuest,LCS,175,216,0.447570,4,1669.105854,234.950221,2023-02-03 23:02:00
25,Cloud9,LCS,581,468,0.553861,-1,1624.383985,220.220726,2023-02-03 23:02:00
28,Evil Geniuses.NA,LCS,174,149,0.538700,2,1562.933973,234.508252,2023-02-03 23:54:00
39,100 Thieves,LCS,193,227,0.459524,3,1497.290893,222.546944,2023-02-04 00:46:00
93,Counter Logic Gaming,LCS,324,344,0.485030,-2,1271.040701,233.928660,2023-02-04 02:34:00
110,TSM,LCS,689,450,0.604917,-2,1231.966284,203.353691,2023-02-04 00:46:00
131,Team Liquid,LCS,490,356,0.579196,2,1187.291203,208.378595,2023-02-04 01:34:00
231,Immortals,LCS,163,160,0.504644,1,1016.593960,237.384131,2023-02-04 02:34:00
307,Golden Guardians,LCS,83,156,0.347280,-5,920.454864,228.267490,2023-02-04 01:34:00
459,Dignitas,LCS,203,272,0.427368,-8,775.133459,268.702049,2023-02-03 23:54:00


In [11]:
rating.loc[(rating['League'].isin(['LCK', 'LPL', 'LEC', 'LCS'])) & (rating['last_game_date'].dt.year == 2023)]

,Team,League,Win,Loss,WinRate,Streak,r,RD,last_game_date
0,Gen.G,LCK,566,386,0.594538,2,2174.436044,125.359783,2023-02-03 12:32:00
1,T1,LCK,857,439,0.661265,1,2104.146209,145.485465,2023-02-04 11:28:00
2,Dplus KIA,LCK,415,172,0.706985,1,2053.279859,139.358524,2023-02-05 10:45:00
4,Liiv SANDBOX,LCK,197,213,0.480488,-2,1955.995677,107.867098,2023-02-03 12:32:00
5,JD Gaming,LPL,381,284,0.572932,3,1949.536544,246.393739,2023-02-02 09:58:00
6,LNG Esports,LPL,389,397,0.494911,6,1924.986802,183.182911,2023-02-05 10:42:00
7,Hanwha Life Esports,LCK,402,410,0.495074,2,1886.433809,121.846096,2023-02-04 07:53:00
8,EDward Gaming,LPL,707,439,0.616928,2,1823.820602,181.522192,2023-02-05 12:10:00
9,KT Rolster,LCK,590,410,0.590000,2,1801.426100,113.106740,2023-02-05 07:14:00
10,Invictus Gaming,LPL,641,482,0.570793,-1,1763.713398,170.616805,2023-02-04 10:59:00


In [12]:
leagues = ['LCK', 'LPL', 'LEC', 'LCS']
for league in leagues:
    print(league)
    lst = rating.loc[(rating['League'] == league) & (rating['last_game_date'].dt.year == 2023), 'Team'].unique()
    print(', '.join(map(lambda x: f"'{x}'", lst)))

LCK
'Gen.G', 'T1', 'Dplus KIA', 'Liiv SANDBOX', 'Hanwha Life Esports', 'KT Rolster', 'Nongshim RedForce', 'Kwangdong Freecs', 'BRION', 'DRX'
LPL
'JD Gaming', 'LNG Esports', 'EDward Gaming', 'Invictus Gaming', 'Top Esports', 'Bilibili Gaming', 'Oh My God', 'Weibo Gaming', 'ThunderTalk Gaming', 'Rare Atom', 'Ninjas in Pyjamas.CN', 'Royal Never Give Up', 'Team WE', 'FunPlus Phoenix', 'Ultra Prime', 'LGD Gaming', 'Anyone's Legend'
LEC
'Team Vitality', 'G2 Esports', 'MAD Lions', 'SK Gaming', 'Astralis', 'Team BDS', 'Team Heretics', 'KOI (Spanish Team)', 'Fnatic', 'Excel Esports'
LCS
'FlyQuest', 'Cloud9', 'Evil Geniuses.NA', '100 Thieves', 'Counter Logic Gaming', 'TSM', 'Team Liquid', 'Immortals', 'Golden Guardians', 'Dignitas'


In [21]:
team_names = {
    'T1': 'T1',
    'GEN': 'Gen.G',
    'DK': 'Dplus KIA',
    'LSB': 'Liiv SANDBOX',
    'KT': 'KT Rolster',
    'HLE': 'Hanwha Life Esports',
    'BRO': 'BRION',
    'DRX': 'DRX',
    'NS': 'Nongshim RedForce',
    'KDF': 'Kwangdong Freecs',
}
lst_team = []
for name in team_names.values():
    team_id = get_team_id(teams_id, name)
    lst_team.append(teams[team_id])

columns = ['Team1', 'Rating1', 'Winprob1', 'Winprob2', 'Rating2', 'Team2']
probs = pd.DataFrame(columns=columns)
for (team1, team2) in permutations(lst_team, 2):
    lst = [
        team1.name, team1.r, team1.get_win_prob(team2) * 100,
        team2.get_win_prob(team1) * 100, team2.r, team2.name
    ]
    df = pd.DataFrame(data=[lst], columns=columns)
    probs = pd.concat([probs, df], ignore_index=True)
probs

,Team1,Rating1,Winprob1,Winprob2,Rating2,Team2
0,T1,2104.146209,40.710204,59.081917,2174.436044,Gen.G
1,T1,2104.146209,56.654973,43.392849,2053.279859,Dplus KIA
2,T1,2104.146209,69.143846,31.555553,1955.995677,Liiv SANDBOX
3,T1,2104.146209,83.755218,17.049922,1801.426100,KT Rolster
4,T1,2104.146209,76.294590,24.272063,1886.433809,Hanwha Life Esports
...,...,...,...,...,...,...
85,Kwangdong Freecs,1719.135733,39.034881,60.622055,1801.426100,KT Rolster
86,Kwangdong Freecs,1719.135733,28.941559,70.623190,1886.433809,Hanwha Life Esports
87,Kwangdong Freecs,1719.135733,55.748086,44.448480,1676.607067,BRION
88,Kwangdong Freecs,1719.135733,55.817581,44.393263,1676.180503,DRX


In [23]:
matches = [
    ('KDF', 'LSB'),
    ('T1', 'BRO'),
    ('DK', 'KT'),
    ('GEN', 'NS'),
    ('BRO', 'HLE'),
    ('LSB', 'DRX'),
    ('NS', 'KDF'),
    ('GEN', 'KT'),
    ('DK', 'HLE'),
    ('T1', 'DRX'),
]

columns = ['Team1', 'Rating1', 'Winprob1', 'Winprob2', 'Rating2', 'Team2']
probs = pd.DataFrame(columns=columns)
for (team1_name, team2_name) in matches:
    team1 = teams[get_team_id(teams_id, team_names[team1_name])]
    team2 = teams[get_team_id(teams_id, team_names[team2_name])]
    lst = [
        team1.name, team1.r, team1.get_win_prob(team2) * 100,
        team2.get_win_prob(team1) * 100, team2.r, team2.name
    ]
    df = pd.DataFrame(data=[lst], columns=columns)
    probs = pd.concat([probs, df], ignore_index=True)
probs

,Team1,Rating1,Winprob1,Winprob2,Rating2,Team2
0,Kwangdong Freecs,1719.135733,21.585632,77.588929,1955.995677,Liiv SANDBOX
1,T1,2104.146209,91.065641,9.670101,1676.607067,BRION
2,Dplus KIA,2053.279859,79.649571,20.983944,1801.426100,KT Rolster
3,Gen.G,2174.436044,91.634779,8.202404,1722.893817,Nongshim RedForce
4,BRION,1676.607067,24.479638,75.757446,1886.433809,Hanwha Life Esports
5,Liiv SANDBOX,1955.995677,82.094330,17.888490,1676.180503,DRX
6,Nongshim RedForce,1722.893817,50.492580,49.501955,1719.135733,Kwangdong Freecs
7,Gen.G,2174.436044,88.297967,11.971687,1801.426100,KT Rolster
8,Dplus KIA,2053.279859,71.008504,29.351624,1886.433809,Hanwha Life Esports
9,T1,2104.146209,91.124931,9.650645,1676.180503,DRX
